In [2]:
import sqlite3
import os

In [3]:
db_location = '/content/gdrive/MyDrive/Database_results'
db_path = os.path.join(db_location, 'summarization_results.db')
print(db_path)

/content/gdrive/MyDrive/Database_results/summarization_results.db


In [4]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [22]:
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

**Calculating cosine_similaity score for Summarization**

In [6]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

In [7]:
def langchain_summarization_cosine_similarity_score(sentence1, sentence2, model_name='bert-large-uncased'):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    tokens1 = tokenizer(sentence1, return_tensors='pt')
    tokens2 = tokenizer(sentence2, return_tensors='pt')

    with torch.no_grad():
        output1 = model(**tokens1)
        output2 = model(**tokens2)

    similarity = F.cosine_similarity(
        output1['last_hidden_state'][0].mean(dim=0).reshape(1, -1),
        output2['last_hidden_state'][0].mean(dim=0).reshape(1, -1),
        dim=1
    ).item()

    return similarity

In [8]:
cursor.execute('''CREATE TABLE IF NOT EXISTS Evaluation_Scores_Summarization_new
          (id INTEGER PRIMARY KEY AUTOINCREMENT,
          claim_number INTEGER,
          claim_name TEXT,
          health_relationship TEXT,
          compound_name TEXT,
          overall_summary TEXT,
          cosine_score FLOAT
           )''')

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.9 MB/s eta 0:00:00


In [ ]:
#cursor.execute('''DROP table Evaluation_Scores_Summarization ''')

In [20]:
cursor.execute('SELECT claim_number, claim_name, compound_name, overall_summary, health_relationship FROM LLM_answers_Prompt3')
entries = cursor.fetchall()

for entry in entries:
    claim_number, claim, compound_name, overall_summary, health_relationship = row[:5]

    cosine_score = langchain_summarization_cosine_similarity_score(str(overall_summary),str(claim))


    cursor.execute('''
      INSERT INTO Evaluation_Scores_Summarization_new (claim_number, claim_name, health_relationship, compound_name, overall_summary,cosine_score)
      VALUES (?, ?, ?,?,?,?)
      ''', (str(claim_number), str(claim), str(health_relationship), str(compound_name),str(overall_summary),cosine_score)
      )

    conn.commit()
    print(f"Data Committed to Database")



Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
Data Committed to Database
D

In [21]:
conn.close()

In [ ]:
#simililarity Function Calling geeting Response from that
